In [1]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import rasterio
from glob import glob
import geopandas as gpd
from scipy.spatial import cKDTree

#path_fire = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\algeria_tunisia.csv"
#df_fire = pd.read_csv(path_fire)
df_fire_clean = pd.read_parquet(r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\local_dataset\dataset\fire_full.parquet", engine="fastparquet")

#df_fire_clean = df_fire[['longitude', 'latitude', 'type']].copy()

print(df_fire_clean.head())

        lon       lat  fire
0   9.48947  31.49290     1
1   9.49053  31.49524     1
2   9.49368  31.49449     1
3   9.49154  31.49420     1
4  10.09115  36.93407     1


In [2]:
print(f"Nombre de lignes dans df_fire_clean : {len(df_fire_clean)}")
print(f"Shape du dataset : {df_fire_clean.shape}")

Nombre de lignes dans df_fire_clean : 56864
Shape du dataset : (56864, 3)


In [3]:
# Créer l'attribut binaire 'fire'
# Si 'type' == 0, 'fire' = 1, sinon 'fire' = 0
"""df_fire_clean['fire'] = np.where(df_fire_clean['type'] == 0, 1, 0)

print(f"df_fire_clean avec colonne 'fire' créé : {df_fire_clean.shape}")
print(df_fire_clean.head())"""

'df_fire_clean[\'fire\'] = np.where(df_fire_clean[\'type\'] == 0, 1, 0)\n\nprint(f"df_fire_clean avec colonne \'fire\' créé : {df_fire_clean.shape}")\nprint(df_fire_clean.head())'

## Chargement et agrégation des données climatiques par saisons + Conversion en DataFrame

In [3]:
climate_csv_path = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\climat_clean.csv"
climate_df = pd.read_csv(climate_csv_path)

print("CSV climat chargé :", climate_df.shape)

# Convertir la date
climate_df["time"] = pd.to_datetime(climate_df["time"])

seasons_indices = {
    's1': [12, 1, 2],   # Hiver (Dec-Jan-Feb)
    's2': [3, 4, 5],    # Printemps
    's3': [6, 7, 8],    # Été
    's4': [9, 10, 11]   # Automne
}

# Ajouter colonne "season"

def get_season(date):
    m = date.month
    if m in [12, 1, 2]:
        return "s1"
    if m in [3, 4, 5]:
        return "s2"
    if m in [6, 7, 8]:
        return "s3"
    return "s4"

climate_df["season"] = climate_df["time"].apply(get_season)

all_dfs = []

variables = ["log_precip",  "tmax","amplitude_thermique"]

for var in variables:
    print(f"\nTraitement de la variable : {var}")

    for season in ["s1", "s2", "s3", "s4"]:
        
        df_season = climate_df[climate_df["season"] == season]

        if df_season.empty:
            print(f"   Saison {season} vide pour {var}")
            continue

        # Agrégation
        if var == "log_precip":
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .sum()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )
        else:
            df_agg = (
                df_season.groupby(["longitude", "latitude"])[var]
                .mean()
                .reset_index()
                .rename(columns={var: f"{var}_{season}"})
            )

        all_dfs.append(df_agg)

        print(f"  → {var}_{season} agrégé ({df_agg.shape[0]} points)")


print("\n Toutes les variables saisonnières climatiques ont été générées.")

CSV climat chargé : (148292, 6)

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax

Traitement de la variable : log_precip
  → log_precip_s1 agrégé (20301 points)
  → log_precip_s2 agrégé (20231 points)
  → log_precip_s3 agrégé (16200 points)
  → log_precip_s4 agrégé (21540 points)

Traitement de la variable : tmax
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé (20231 points)
  → tmax_s3 agrégé (16200 points)
  → tmax_s4 agrégé (21540 points)

Traitement de la variable : amplitude_thermique
  → amplitude_thermique_s1 agrégé (20301 points)
  → amplitude_thermique_s2 agrégé (20231 points)
  → amplitude_thermique_s3 agrégé (16200 points)
  → amplitude_thermique_s4 agrégé (21540 points)

 Toutes les variables saisonnières climatiques ont été générées.
  → tmax_s1 agrégé (20301 points)
  → tmax_s2 agrégé 

## Fusion des DataFrames climatiques en un seul DataFrame

In [4]:
# Commencer avec le premier DataFrame
df_climat_merged = all_dfs[0]

# Fusionner tous les autres DataFrames sur longitude et latitude
for i, df in enumerate(all_dfs[1:], 1):
    df_climat_merged = pd.merge(df_climat_merged, df, on=['longitude', 'latitude'], how='inner')
    print(f"   → Fusion {i}/{len(all_dfs)-1} terminée")

print(f"\n DataFrame climatique fusionné créé : {df_climat_merged.shape}")
print(f" Colonnes : {list(df_climat_merged.columns)}")
print("\n Aperçu :")
print(df_climat_merged.head())

   → Fusion 1/11 terminée
   → Fusion 2/11 terminée
   → Fusion 3/11 terminée
   → Fusion 4/11 terminée
   → Fusion 5/11 terminée
   → Fusion 6/11 terminée
   → Fusion 7/11 terminée
   → Fusion 8/11 terminée
   → Fusion 9/11 terminée
   → Fusion 10/11 terminée
   → Fusion 11/11 terminée

 DataFrame climatique fusionné créé : (10113, 14)
 Colonnes : ['longitude', 'latitude', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4']

 Aperçu :
   longitude   latitude  log_precip_s1  log_precip_s2  log_precip_s3  \
0     -8.625  27.291667       3.082025       0.916291       0.832909   
1     -8.625  27.375000       3.224341       0.916291       1.964311   
2     -8.625  27.458333       3.655782       1.321756       1.996060   
3     -8.625  27.541667       3.654403       1.267652       1.163151   
4     -8.625  27.625000       2.0149

## Merge final : Fire + Climat (approche optimisée avec pd.merge)

In [5]:
# Merge Fire + Climat avec KDTree

# Extraire les coordonnées des points de feu
fire_coords = df_fire_clean[["lon", "lat"]].to_numpy()
print(f"Fire coords shape: {fire_coords.shape}")

# Extraire les coordonnées de la grille climatique
climat_coords = df_climat_merged[["longitude", "latitude"]].to_numpy()
print(f"Climat coords shape: {climat_coords.shape}")

# Construire le KDTree sur la grille climatique
print("Construction du KDTree...")
tree = cKDTree(climat_coords)
print("KDTree construit.")

# Trouver le point de grille le plus proche pour chaque point de feu
print("Recherche des plus proches voisins...")
dist, idx = tree.query(fire_coords, k=1)
print("Plus proches voisins trouvés.")

print(f"Distance moyenne: {dist.mean():.6f}°")
print(f"Distance max: {dist.max():.6f}°")
print(f"Distance min: {dist.min():.6f}°")

# Fusionner les données climatiques avec les données de feu
df_final = pd.concat([
    df_fire_clean.reset_index(drop=True),
    df_climat_merged.drop(columns=["longitude", "latitude"]).iloc[idx.flatten()].reset_index(drop=True)
], axis=1)

print(f"\nDataset final après merge : {df_final.shape}")
print(f"Colonnes : {list(df_final.columns)}")

# Vérifier les valeurs manquantes
climatic_cols = [col for col in df_final.columns if any(season in col for season in ['s1', 's2', 's3', 's4'])]
print(f"\nValeurs manquantes dans les variables climatiques :")
print(df_final[climatic_cols].isnull().sum())

# Vérifier le pourcentage de correspondances
matches_count = df_final[climatic_cols[0]].notna().sum()
total_points = len(df_final)
print(f"\nPoints avec données climatiques : {matches_count}/{total_points} ({matches_count/total_points*100:.1f}%)")

Fire coords shape: (56864, 2)
Climat coords shape: (10113, 2)
Construction du KDTree...
KDTree construit.
Recherche des plus proches voisins...
Plus proches voisins trouvés.
Distance moyenne: 0.307297°
Distance max: 2.171342°
Distance min: 0.000970°

Dataset final après merge : (56864, 15)
Colonnes : ['lon', 'lat', 'fire', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4']

Valeurs manquantes dans les variables climatiques :
log_precip_s1             0
log_precip_s2             0
log_precip_s3             0
log_precip_s4             0
tmax_s1                   0
tmax_s2                   0
tmax_s3                   0
tmax_s4                   0
amplitude_thermique_s1    0
amplitude_thermique_s2    0
amplitude_thermique_s3    0
amplitude_thermique_s4    0
dtype: int64

Points avec données climatiques : 56864/56864 (100.0%)


In [6]:
# Charger landcover avec les polygones originaux
gdf_land = gpd.read_file(r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\data\landcover_final_ML.gpkg")

print(f"Land cover chargé : {gdf_land.shape[0]} polygones")
print(f"CRS : {gdf_land.crs}")
print(f"Colonnes : {list(gdf_land.columns)}")

c:\Users\anfel\anaconda3\envs\dm\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


Land cover chargé : 307385 polygones
CRS : EPSG:4326
Colonnes : ['GRIDCODE', 'pays', 'log_area_sqm', 'lcc_code_encoded', 'geometry']


In [7]:
# Convertir df_final en GeoDataFrame avec des points géométriques
gdf_final = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final['lon'], df_final['lat']),
    crs="EPSG:4326"  # WGS84 (système de coordonnées standard)
)

print(f"\nGeoDataFrame créé : {gdf_final.shape}")
print(f"CRS : {gdf_final.crs}")
print(f"Type de géométrie : {gdf_final.geometry.geom_type.unique()}")


GeoDataFrame créé : (56864, 16)
CRS : EPSG:4326
Type de géométrie : ['Point']


In [8]:
# Aligner les systèmes de coordonnées si nécessaire
if gdf_final.crs != gdf_land.crs:
    print(f"\nReprojection de gdf_land : {gdf_land.crs} → {gdf_final.crs}")
    gdf_land = gdf_land.to_crs(gdf_final.crs)
else:
    print(f"\nLes CRS sont déjà alignés : {gdf_final.crs}")


Les CRS sont déjà alignés : EPSG:4326


In [9]:
# 1. Vérifier les bounding boxes
print("\n Points (Fire + Climat):")
points_bounds = gdf_final.total_bounds
print(f"   Min Lon: {points_bounds[0]:.4f}, Max Lon: {points_bounds[2]:.4f}")
print(f"   Min Lat: {points_bounds[1]:.4f}, Max Lat: {points_bounds[3]:.4f}")

print("\n  Polygones (Land Cover):")
land_bounds = gdf_land.total_bounds
print(f"   Min Lon: {land_bounds[0]:.4f}, Max Lon: {land_bounds[2]:.4f}")
print(f"   Min Lat: {land_bounds[1]:.4f}, Max Lat: {land_bounds[3]:.4f}")

# 2. Vérifier si les zones se chevauchent
lon_overlap = not (points_bounds[2] < land_bounds[0] or points_bounds[0] > land_bounds[2])
lat_overlap = not (points_bounds[3] < land_bounds[1] or points_bounds[1] > land_bounds[3])

if lon_overlap and lat_overlap:
    print(" Les zones SE CHEVAUCHENT ")
else:
    print(" Les zones NE SE CHEVAUCHENT PAS ")
    if not lon_overlap:
        print(" Pas de chevauchement en LONGITUDE")
    if not lat_overlap:
        print("Pas de chevauchement en LATITUDE")

# 3. Échantillon de points
print("\n ÉCHANTILLON DE POINTS (5 premiers)")
print(gdf_final[['lon', 'lat', 'geometry']].head())

# 4. Échantillon de polygones
print("\n ÉCHANTILLON DE POLYGONES (5 premiers)")
print(gdf_land[['GRIDCODE', 'geometry']].head())

# 5. Vérifier les géométries invalides
print("\n VÉRIFICATION DE LA VALIDITÉ DES GÉOMÉTRIES")
invalid_points = (~gdf_final.is_valid).sum()
invalid_polys = (~gdf_land.is_valid).sum()
print(f"   Points invalides : {invalid_points}/{len(gdf_final)}")
print(f"   Polygones invalides : {invalid_polys}/{len(gdf_land)}")

if invalid_points > 0 or invalid_polys > 0:
    print("    Il y a des géométries invalides !")
else:
    print("    Toutes les géométries sont valides")


 Points (Fire + Climat):
   Min Lon: -8.1280, Max Lon: 11.1119
   Min Lat: 19.5932, Max Lat: 37.3332

  Polygones (Land Cover):
   Min Lon: -8.6722, Max Lon: 11.9681
   Min Lat: 18.9660, Max Lat: 37.5439
 Les zones SE CHEVAUCHENT 

 ÉCHANTILLON DE POINTS (5 premiers)
        lon       lat                   geometry
0   9.48947  31.49290    POINT (9.48947 31.4929)
1   9.49053  31.49524   POINT (9.49053 31.49524)
2   9.49368  31.49449   POINT (9.49368 31.49449)
3   9.49154  31.49420    POINT (9.49154 31.4942)
4  10.09115  36.93407  POINT (10.09115 36.93407)

 ÉCHANTILLON DE POLYGONES (5 premiers)
   GRIDCODE                                           geometry
0       210  POLYGON ((6.41528 37.08696, 6.43103 37.0855, 6...
1       210  POLYGON ((7.37137 37.08194, 7.3709 37.08717, 7...
2        50  POLYGON ((6.12361 36.68472, 6.12361 36.69306, ...
3       130  POLYGON ((6.44583 37.07917, 6.44583 37.08194, ...
4        50  POLYGON ((6.40694 37.08194, 6.40694 37.08472, ...

 VÉRIFICATION DE L

In [10]:
BUFFER_SIZE = 0.005  # 500 mètres

# Créer une copie avec buffer
gdf_final_buffered = gdf_final.copy()
gdf_final_buffered['geometry'] = gdf_final_buffered.geometry.buffer(BUFFER_SIZE)

# Spatial join sur tout le dataset
gdf_full = gpd.sjoin(
    gdf_final_buffered,
    gdf_land[['geometry', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']],
    how="left",
    predicate="intersects"
)

# Statistiques
total_rows = len(gdf_full)
unique_points = gdf_full.index.nunique()
matches = gdf_full['GRIDCODE'].notna().sum()
unique_matches = gdf_full[gdf_full['GRIDCODE'].notna()].index.nunique()


print(f"\n RÉSULTATS :")
print(f"   • Total de lignes après join : {total_rows:,}")
print(f"   • Points uniques traités : {unique_points:,}")
print(f"   • Lignes avec correspondance : {matches:,}")
print(f"   • Points uniques ayant trouvé un match : {unique_matches:,}")
print(f"   • Taux de couverture : {(unique_matches/unique_points)*100:.1f}%")

# Vérifier les duplications
duplications = total_rows - unique_points
if duplications > 0:
    print(f"\n  {duplications:,} lignes dupliquées détectées")
else:
    print(f"\n Aucune duplication ")

C:\Users\anfel\AppData\Local\Temp\ipykernel_9156\1355669867.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_final_buffered['geometry'] = gdf_final_buffered.geometry.buffer(BUFFER_SIZE)



 RÉSULTATS :
   • Total de lignes après join : 66,426
   • Points uniques traités : 56,864
   • Lignes avec correspondance : 28,215
   • Points uniques ayant trouvé un match : 18,653
   • Taux de couverture : 32.8%

  9,562 lignes dupliquées détectées


In [11]:
# Supprimer les duplications (garder la première correspondance pour chaque point)
if len(gdf_full) > len(gdf_final):
    print(f"   Avant dé-duplication : {len(gdf_full):,} lignes")
    gdf_full = gdf_full[~gdf_full.index.duplicated(keep='first')]
    print(f"   Après dé-duplication : {len(gdf_full):,} lignes")
    removed = len(gdf_full) - len(gdf_final)
    if removed != 0:
        print(f"Duplications supprimées")
else:
    print(" Aucune duplication à nettoyer")

# Nettoyage final : supprimer les colonnes inutiles
df_full = gdf_full.drop(columns=['geometry', 'index_right'], errors='ignore')
df_full = pd.DataFrame(df_full)

print(f"\nDATASET FINAL :")
print(f"   Shape : {df_full.shape}")
print(f"   Colonnes : {list(df_full.columns)}")

# Statistiques finales
land_cover_matches = df_full['GRIDCODE'].notna().sum()
total_points = len(df_full)
print(f"\n COUVERTURE LAND COVER :")
print(f"   Points avec land cover : {land_cover_matches:,}/{total_points:,} ({(land_cover_matches/total_points)*100:.1f}%)")
print(f"   Points sans land cover : {total_points - land_cover_matches:,} ({((total_points - land_cover_matches)/total_points)*100:.1f}%)")

print("\n" + "="*70)

   Avant dé-duplication : 66,426 lignes
   Après dé-duplication : 56,864 lignes

DATASET FINAL :
   Shape : (56864, 18)
   Colonnes : ['lon', 'lat', 'fire', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']

 COUVERTURE LAND COVER :
   Points avec land cover : 18,653/56,864 (32.8%)
   Points sans land cover : 38,211 (67.2%)



In [12]:
# Pour les points sans match avec buffer, utiliser KDTree
print("\n TRAITEMENT DES POINTS SANS MATCH AVEC KD-TREE")

# Identifier les points sans land cover après le buffer
mask_no_match = df_full['GRIDCODE'].isna()
points_sans_match = mask_no_match.sum()

print(f"   • Points sans match après buffer : {points_sans_match:,}")

if points_sans_match > 0:
    # Extraire les coordonnées des points sans match
    gdf_no_match = gdf_final[mask_no_match].copy()
    no_match_coords = gdf_no_match[['lon', 'lat']].to_numpy()
    
    # Extraire les centroïdes des polygones land cover
    land_centroids = gdf_land.geometry.centroid
    land_coords = np.array([[geom.x, geom.y] for geom in land_centroids])
    
    # Construire le KDTree sur les centroïdes
    print("   • Construction du KDTree sur les centroïdes...")
    tree = cKDTree(land_coords)
    
    # Trouver le polygone le plus proche pour chaque point sans match
    print("   • Recherche des plus proches polygones...")
    dist, idx = tree.query(no_match_coords, k=1)
    
    print(f"   • Distance moyenne: {dist.mean():.6f}° ({dist.mean() * 111:.2f} km)")
    print(f"   • Distance max: {dist.max():.6f}° ({dist.max() * 111:.2f} km)")
    
    # Récupérer les attributs land cover correspondants
    land_attrs = gdf_land[['GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']].iloc[idx.flatten()].reset_index(drop=True)
    
    # Mettre à jour df_full pour les points sans match
    df_full.loc[mask_no_match, 'GRIDCODE'] = land_attrs['GRIDCODE'].values
    df_full.loc[mask_no_match, 'log_area_sqm'] = land_attrs['log_area_sqm'].values
    df_full.loc[mask_no_match, 'lcc_code_encoded'] = land_attrs['lcc_code_encoded'].values
    
    # Ajouter la colonne de distance pour traçabilité
    df_full['distance_to_landcover_deg'] = np.nan
    df_full.loc[mask_no_match, 'distance_to_landcover_deg'] = dist
    
    print(f"   • Points complétés avec KDTree : {points_sans_match:,}")
else:
    print("   • Tous les points ont trouvé un match avec le buffer !")

# Statistiques finales
print(f"\n RÉSULTATS FINAUX (BUFFER + KD-TREE) :")
print(f"   • Dataset shape : {df_full.shape}")
print(f"   • Points avec land cover : {df_full['GRIDCODE'].notna().sum():,} ({(df_full['GRIDCODE'].notna().sum()/len(df_full))*100:.1f}%)")
print(f"   • Points sans land cover : {df_full['GRIDCODE'].isna().sum():,}")



 TRAITEMENT DES POINTS SANS MATCH AVEC KD-TREE
   • Points sans match après buffer : 38,211


C:\Users\anfel\AppData\Local\Temp\ipykernel_9156\2970428372.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  land_centroids = gdf_land.geometry.centroid


   • Construction du KDTree sur les centroïdes...
   • Recherche des plus proches polygones...
   • Distance moyenne: 0.057897° (6.43 km)
   • Distance max: 1.086991° (120.66 km)
   • Points complétés avec KDTree : 38,211

 RÉSULTATS FINAUX (BUFFER + KD-TREE) :
   • Dataset shape : (56864, 19)
   • Points avec land cover : 56,864 (100.0%)
   • Points sans land cover : 0
   • Distance moyenne: 0.057897° (6.43 km)
   • Distance max: 1.086991° (120.66 km)
   • Points complétés avec KDTree : 38,211

 RÉSULTATS FINAUX (BUFFER + KD-TREE) :
   • Dataset shape : (56864, 19)
   • Points avec land cover : 56,864 (100.0%)
   • Points sans land cover : 0


In [14]:
# Supprimer la colonne de distance si non nécessaire (optionnel)
df_full = df_full.drop(columns=['distance_to_landcover_deg'], errors='ignore')

# Enregistrer le dataset final en format Parquet
output_path = r"C:\Users\anfel\OneDrive\Desktop\M2\prjt\local_dataset\dataset\buffer_kdtree.parquet"

df_full.to_parquet(output_path, engine='fastparquet', compression='snappy')

print(f"\n Dataset sauvegardé : {output_path}")
print(f"   • Shape finale : {df_full.shape}")
print(f"   • Colonnes : {list(df_full.columns)}")



 Dataset sauvegardé : C:\Users\anfel\OneDrive\Desktop\M2\prjt\local_dataset\dataset\buffer_kdtree.parquet
   • Shape finale : (56864, 18)
   • Colonnes : ['lon', 'lat', 'fire', 'log_precip_s1', 'log_precip_s2', 'log_precip_s3', 'log_precip_s4', 'tmax_s1', 'tmax_s2', 'tmax_s3', 'tmax_s4', 'amplitude_thermique_s1', 'amplitude_thermique_s2', 'amplitude_thermique_s3', 'amplitude_thermique_s4', 'GRIDCODE', 'log_area_sqm', 'lcc_code_encoded']
